In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver

In [2]:
# <div class="race-list">
# 		<a href="/calendar/2023-2024/race-3-4h-of-dubai" style="text-decoration:none!important;">
#           <!--<div class="race-track-img"  style="background-image: url('../assets/track-sepang.svg')"></div>-->
#           <div class="race-track"><span class="nat"><img src="../assets/flags/united-arab-emirates.png"></span>Dubai, United Arab Emirates</div>
#           <div class="race-date">4 FEB 2024</div>
#           <div class="race-circuit">Dubai Autodrome</div>
# 		</a>
#         </div>

In [3]:

driver = webdriver.Chrome()
driver.get("https://www.asianlemansseries.com/calendar/")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('div', {'class': 'race-list'})

driver.quit()


In [4]:
schedule_details

[<div class="race-list">
 <a href="/calendar/2023-2024/race-1-4h-of-sepang" style="text-decoration:none!important;">
 <!--<div class="race-track-img"  style="background-image: url('../assets/track-chang.svg')"></div>-->
 <div class="race-track"><span class="nat"><img src="../assets/flags/malaysia.png"/></span>Sepang, Malaysia</div>
 <div class="race-date">2 DEC 2023</div>
 <div class="race-circuit">Sepang International Circuit</div>
 </a>
 </div>,
 <div class="race-list">
 <a href="/calendar/2023-2024/race-2-4h-of-sepang" style="text-decoration:none!important;">
 <!--<div class="race-track-img"  style="background-image: url('../assets/track-chang.svg')"></div>-->
 <div class="race-track"><span class="nat"><img src="../assets/flags/malaysia.png"/></span>Sepang, Malaysia</div>
 <div class="race-date">3 DEC 2023</div>
 <div class="race-circuit">Sepang International Circuit</div>
 </a>
 </div>,
 <div class="race-list">
 <a href="/calendar/2023-2024/race-3-4h-of-dubai" style="text-decoratio

In [5]:
days = [] # <div class="race-date">11 FEB 2024</div>
months = []
titles = [] #<a href="/calendar/2023-2024/race-5-4h-of-abu-dhabi" style="text-decoration:none!important;">
descs = []
circuits = [] # <div class="race-circuit">Yas Marina Circuit</div>


month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12,
        'DEC': 12
    }

round_counter = 1

for i in schedule_details:
    temp_circuit = i.find('div', {'class': 'race-circuit'}).text
    circuits.append(temp_circuit)
    
    temp_date = i.find('div', {'class':"race-date"}).text
    temp_date = temp_date.split(' ')
    days.append(int(temp_date[0]))
    
    temp_month = temp_date[1]
    converted_month = month_mapping[temp_month]
    months.append(converted_month)
    
    
    temp_title = 'AsLMS Round' + ' ' + str(round_counter)
    round_counter += 1
    titles.append(temp_title)
    
    descs.append('4 Hours')
    

 
descs

['4 Hours', '4 Hours', '4 Hours', '4 Hours', '4 Hours']

In [6]:
type(days[0])

int

In [7]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [8]:
AsLMS = Calendar()
AsLMS.add('prodid', '-//My calendar product//example.com//')
AsLMS.add('version', '2.0')

In [9]:


uid = 110 
for title, circuit, day, month, desc in zip(titles, circuits, days, months, descs):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(2024, month, day, 4, 0, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(2024, month, day, 8, 0, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(2024, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    AsLMS.add_component(ievent)

In [10]:
AsLMS

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'AsLMS Round 1''), 'DESCRIPTION': vText('b'4 Hours''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001B2ABB222B0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001B2ABB22220>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001B2ABB22190>, 'LOCATION': vText('b'Sepang International Circuit''), 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'AsLMS Round 2''), 'DESCRIPTION': vText('b'4 Hours''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001B2ABB1C580>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001B2ABB1CE20>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001B2ABB1CF70>, 'LOCATION': vText('b'Sepang International Circuit''), 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'AsLMS Round 3''), 'DESCRIPTION': vText('b'4 Hours''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001B2ABB1CDC0>, 'DTEND': <icalendar.p

In [11]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'AsLMS.ics'), 'wb')
f.write(AsLMS.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
